# 04-Spam-Classifier

It's time to make our first real Machine Learning application of NLP: a spam classifier!

A spam classifier is a Machine Learning model that classifier texts (email or SMS) into two categories: Spam (1) or legitimate (0).

To do that, we will reuse our knowledge: we will apply preprocessing and BOW (Bag Of Words) on a dataset of texts.
Then we will use a classifier to predict to which class belong a new email/SMS, based on the BOW.

First things first: import the needed libraries.

In [12]:
# Import NLTK and all the needed libraries
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

Load now the dataset in *spam.csv* using pandas. Use the 'latin-1' encoding as loading option.

In [13]:
# TODO: Load the dataset 
df = pd.read_csv('spam.csv', encoding='latin-1')
df.head()

,Class,Message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


As usual, I suggest you to explore a bit this dataset.

In [3]:
# TODO: explore the dataset


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Class    5572 non-null   object
 1   Message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


So as you see we have a column containing the labels, and a column containing the text to classify.

We will begin by doing the usual preprocessing: tokenization, punctuation removal and lemmatization.

In [34]:
# TODO: Perform preprocessing over all the text
pd.set_option("display.max_colwidth", 10000)
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
lemFilter = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
arr = [tokenizer.tokenize(lemFilter.lemmatize(item)) for item in df['Message']]
arr

[['Go',
  'until',
  'jurong',
  'point',
  'crazy',
  'Available',
  'only',
  'in',
  'bugis',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  'Cine',
  'there',
  'got',
  'amore',
  'wat'],
 ['Ok', 'lar', 'Joking', 'wif', 'u', 'oni'],
 ['Free',
  'entry',
  'in',
  '2',
  'a',
  'wkly',
  'comp',
  'to',
  'win',
  'FA',
  'Cup',
  'final',
  'tkts',
  '21st',
  'May',
  '2005',
  'Text',
  'FA',
  'to',
  '87121',
  'to',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'T',
  'C',
  's',
  'apply',
  '08452810075over18',
  's'],
 ['U', 'dun', 'say', 'so', 'early', 'hor', 'U', 'c', 'already', 'then', 'say'],
 ['Nah',
  'I',
  'don',
  't',
  'think',
  'he',
  'goes',
  'to',
  'usf',
  'he',
  'lives',
  'around',
  'here',
  'though'],
 ['FreeMsg',
  'Hey',
  'there',
  'darling',
  'it',
  's',
  'been',
  '3',
  'week',
  's',
  'now',
  'and',
  'no',
  'word',
  'back',
  'I',
  'd',
  'like',
  'some',
  'fun',
  'you',
  'up',
  'for',
  'it',


Ok now we have our preprocessed data. Next step is to do a BOW.

In [40]:
# TODO: compute the BOW
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000,stop_words='english')
BOW = vectorizer.fit_transform(df['Message']).toarray()
print (BOW)
tokens=vectorizer.get_feature_names()
print(tokens)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['00', '000', '03', '04', '0800', '08000839402', '08000930705', '08712460324', '10', '100', '10p', '11', '11mths', '12', '12hrs', '150', '150p', '150ppm', '16', '18', '1st', '20', '2003', '2004', '20p', '2day', '2lands', '2nd', '2nite', '30', '4u', '50', '500', '5we', '750', '800', '8007', '86688', '87066', '87077', 'abiola', 'able', 'abt', 'ac', 'accept', 'access', 'account', 'actually', 'address', 'admirer', 'aft', 'afternoon', 'age', 'age16', 'ago', 'ah', 'aight', 'al', 'alright', 'amp', 'angry', 'ans', 'answer', 'anytime', 'apply', 'ar', 'ard', 'area', 'asap', 'ask', 'askd', 'asked', 'asking', 'ass', 'attempt', 'auction', 'available', 'ave', 'await', 'awaiting', 'award', 'awarded', 'away', 'awesome', 'b4', 'babe', 'baby', 'bad', 'balance', 'bank', 'bath', 'bathe', 'bb', 'bcoz', 'beautiful', 'bed', 'believe', 'best', 'better', 'big', 'birthday', 'bit', 'blood', 'bl

Then make a new dataframe as usual to have a visual idea of the words used and their frequencies.

In [43]:
# TODO: Make a new dataframe with the BOW
bow_df = pd.DataFrame(data=BOW, columns=tokens)
bow_df

'ur'

Let's check what is the most used word in the spam category and the non spam category.

There are two steps: first add the class to the BOW dataframe. Second, filter on a class, sum all the values and print the most frequent one.

In [13]:
# TODO: print the most used word in the spam and non spam category


most frequent spam word: free
most frequent non spam word: im


You should find that the most frequent spam word is 'free', not so surprising, right?

Now we can make a classifier based on our BOW. We will use a simple logistic regression here for the example.

You're an expert, you know what to do, right? Split the data, train your model, predict and see the performance.

In [20]:
# TODO: Perform a classification to predict whether a message is a spam or not


[0 0 1 ... 0 0 0]


0.9748878923766816

What precision do you get? Check by hand on some samples where it did predict well to check what could go wrong...

Try to use other models and try to improve your results.